In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import src.read_weather as rw
import psycopg2 as pg2
import sqlalchemy
import julian
import datetime
import s3fs
from collections import defaultdict

In [2]:
import gdal

In [3]:
txt_path = 'data/weather/meta_data/ghcnd-stations.txt'
path_2000= 'data/weather/gz_files/2017.csv.gz'

In [17]:
meta_data=rw.read_metadata_txt(txt_path)

In [18]:
idi = rw.station_id_lookup(meta_data)

In [19]:
idi['CA004018880']

array([50.35, -102.2667, 598.0, 'SK'], dtype=object)

In [87]:
weather_data=rw.read_weather_data(path_2000)

In [94]:
path_to_save= 'data/weather/clean_csv/test.csv'

In [92]:
test= weather_data.head(20)

In [95]:
rw.make_clean_csv(test, path_to_save)

True

In [99]:
weather_data

,station_id,measurement_date,measurement_type,measurement_flag
0,CA1MB000296,2017-01-01,PRCP,0
1,US1MNCV0008,2017-01-01,PRCP,0
2,US1MNCV0008,2017-01-01,SNOW,0
3,US1MISW0005,2017-01-01,PRCP,0
4,US1MISW0005,2017-01-01,SNOW,0
5,US1MISW0005,2017-01-01,SNWD,0
6,ASN00015643,2017-01-01,TMAX,274
7,ASN00015643,2017-01-01,TMIN,218
8,ASN00015643,2017-01-01,PRCP,2
9,US1MAMD0069,2017-01-01,PRCP,56


In [ ]:
CREATE TABLE weather_data 
(index int,
 station_id text, 
 measurement_date date, 
 measurement_type text, 
 measurement_flag int); 

In [ ]:
COPY weather_data FROM '/Users/Berzyy/plant_forecast/data/weather/clean_csv/clean_2000.csv' 
WITH (FORMAT csv);

In [ ]:
copy weather_data FROM '/Users/Berzyy/plant_forecast/data/weather/clean_csv/clean_2000.csv' DELIMITER ',' CSV HEADER FALSE;

In [ ]:
COPY weather_data from '/Users/Berzyy/plant_forecast/data/weather/clean_csv/clean_2000.csv' csv header;

In [10]:
weather_data.head()

,station_id,measurement_date,measurement_type,measurement_flag


In [18]:
meta_data.head()

,station_id,latitude,longitude,elevation,state
0,ACW00011604,17.1167,-61.7833,10.1,ST
1,ACW00011647,17.1333,-61.7833,19.2,ST
2,AE000041196,25.3330,55.5170,34.0,SHARJAH
3,AEM00041194,25.2550,55.3640,10.4,DUBAI
4,AEM00041217,24.4330,54.6510,26.8,ABU
